In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pickle
from datetime import datetime

In [16]:
%%time
test_df = pd.read_csv("./data/test_df3.csv")

CPU times: user 2.12 s, sys: 695 ms, total: 2.82 s
Wall time: 3.25 s


In [17]:
pd.options.display.float_format = "{:.2f}".format
# test_df = test_df.head(998)
test_df

,index,TIMESTAMP_x,BUSROUTE_ID,BUS_ID,BUSSTOP_ID,BUSSTOP_SEQ,RECORD_DATE,TIMESTAMP_y,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_BK,PRED
0,3220254,1579685016.00,11100010,2054,11,3,2020-01-22 09:23:36,1579888192.09,0.00,1.49,38.00,1579685016.00,168.09
1,3220255,1579685137.48,11100010,2054,9,4,0,1579685127.33,0.00,1.49,38.00,0.00,111.33
2,3220256,1579685246.00,11100010,2054,6,5,2020-01-22 09:27:26,1579685226.78,0.00,1.49,38.00,1579685246.00,99.47
3,3220257,1579685333.59,11100010,2054,4,6,0,1579685337.48,0.00,1.49,38.00,0.00,91.48
4,3220258,1579685473.01,11100010,2054,147,7,0,1579685483.07,0.00,1.49,38.00,0.00,145.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1557459,4777713,1579883967.00,12100232,740518021,657,9,2020-01-24 16:39:27,1579883543.64,43975.00,1.88,8.00,1579883967.00,144.21
1557460,4777714,1579886880.00,12100232,740518021,219,2,2020-01-24 17:28:00,1579884096.41,43976.00,1.88,4.00,1579886880.00,129.41
1557461,4777715,1579887228.97,12100232,740518021,221,3,0,1579886997.28,43976.00,1.88,4.00,0.00,117.28
1557462,4777716,1579887668.62,12100232,740518021,242,4,0,1579887145.04,43976.00,1.88,4.00,0.00,148.84


In [18]:
%%time
gap_list = []
for idx, row in test_df.iterrows():
    timestamp = row['TIMESTAMP_BK']
    if (timestamp == 0):
        prev_timestamp = test_df.iloc[idx-1]['TIMESTAMP_BK']
        if (prev_timestamp != 0):
            current_timestamp = prev_timestamp + row['PRED']
            gap_start = idx-1
            pass
        else:
            current_timestamp += row['PRED']
            
        test_df.at[idx, 'TIMESTAMP_NEW'] = current_timestamp
        
        next_timestamp = test_df.iloc[idx+1]['TIMESTAMP_BK']
        if (next_timestamp!=0):
            gap_end = idx+1
            gap_list.append((gap_start, gap_end))
            
            test_df.at[idx+1, 'TIMESTAMP_NEW'] = current_timestamp + test_df.iloc[idx+1]['PRED']
            #cnt = cnt + 1
            #if cnt>5:
            #    break
            
len(gap_list)

CPU times: user 14min 10s, sys: 6 s, total: 14min 16s
Wall time: 14min 45s


250529

In [19]:
gap_list[:5]

[(0, 2), (2, 5), (5, 8), (8, 13), (13, 18)]

In [24]:
# test_df_bk = test_df
# test_df = test_df.head(998)
test_df = test_df_bk
test_df

,index,TIMESTAMP_x,BUSROUTE_ID,BUS_ID,BUSSTOP_ID,BUSSTOP_SEQ,RECORD_DATE,TIMESTAMP_y,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_BK,PRED,TIMESTAMP_NEW
0,3220254,1579685016.00,11100010,2054,11,3,2020-01-22 09:23:36,1579888192.09,0.00,1.49,38.00,1579685016.00,168.09,1579685016.00
1,3220255,1579685137.48,11100010,2054,9,4,0,1579685127.33,0.00,1.49,38.00,0.00,111.33,1579685127.33
2,3220256,1579685246.00,11100010,2054,6,5,2020-01-22 09:27:26,1579685226.78,0.00,1.49,38.00,1579685246.00,99.47,1579685226.81
3,3220257,1579685333.59,11100010,2054,4,6,0,1579685337.48,0.00,1.49,38.00,0.00,91.48,1579685337.48
4,3220258,1579685473.01,11100010,2054,147,7,0,1579685483.07,0.00,1.49,38.00,0.00,145.26,1579685482.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1557459,4777713,1579883967.00,12100232,740518021,657,9,2020-01-24 16:39:27,1579883543.64,43975.00,1.88,8.00,1579883967.00,144.21,1579883545.12
1557460,4777714,1579886880.00,12100232,740518021,219,2,2020-01-24 17:28:00,1579884096.41,43976.00,1.88,4.00,1579886880.00,129.41,nan
1557461,4777715,1579887228.97,12100232,740518021,221,3,0,1579886997.28,43976.00,1.88,4.00,0.00,117.28,1579886997.28
1557462,4777716,1579887668.62,12100232,740518021,242,4,0,1579887145.04,43976.00,1.88,4.00,0.00,148.84,1579887146.13


In [22]:
%%time
# pd.options.display.float_format = "{:.2f}".format
# cnt = 0

for gap in gap_list:
    gap_start     = gap[0]
    gap_end       = gap[1]
    scale_min     = test_df.iloc[gap_start]['TIMESTAMP_BK']
    scale_max     = test_df.iloc[gap_end  ]['TIMESTAMP_BK']
    bk_timestamps = test_df.iloc[gap_start:gap_end+1]['TIMESTAMP_NEW'].values
    
    bk_timestamps[0] = scale_min
    
    rescaled = np.interp(
        bk_timestamps, 
        (bk_timestamps.min(), bk_timestamps.max()), 
        (scale_min, scale_max)
    )
    iter_count = gap_end - gap_start
    for i in range(0, iter_count+1):
        idx = gap_start+i
        test_df.at[idx, 'TIMESTAMP_SCALED'] = rescaled[i]
        test_df.at[idx, 'DATETIME_SCALED' ] = datetime.utcfromtimestamp(rescaled[i])
        
#     cnt = cnt+1
#     if cnt>3:
#        break

CPU times: user 6.64 ms, sys: 299 µs, total: 6.94 ms
Wall time: 7.04 ms


In [23]:
test_df.head(60)

,index,TIMESTAMP_x,BUSROUTE_ID,BUS_ID,BUSSTOP_ID,BUSSTOP_SEQ,RECORD_DATE,TIMESTAMP_y,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_BK,PRED,TIMESTAMP_NEW,TIMESTAMP_SCALED,DATETIME_SCALED
0,3220254,1579685016.00,11100010,2054,11,3,2020-01-22 09:23:36,1579888192.09,0.00,1.49,38.00,1579685016.00,168.09,1579685016.00,1579685016.00,2020-01-22 09:23:36.000000
1,3220255,1579685137.48,11100010,2054,9,4,0,1579685127.33,0.00,1.49,38.00,0.00,111.33,1579685127.33,1579685127.33,2020-01-22 09:25:27.331718
2,3220256,1579685246.00,11100010,2054,6,5,2020-01-22 09:27:26,1579685226.78,0.00,1.49,38.00,1579685246.00,99.47,1579685246.00,1579685246.00,2020-01-22 09:27:26.000000
3,3220257,1579685333.59,11100010,2054,4,6,0,1579685337.48,0.00,1.49,38.00,0.00,91.48,1579685337.48,1579685337.48,2020-01-22 09:28:57.475239
4,3220258,1579685473.01,11100010,2054,147,7,0,1579685483.07,0.00,1.49,38.00,0.00,145.26,1579685482.73,1579685482.73,2020-01-22 09:31:22.733288
5,3220259,1579685572.00,11100010,2054,149,8,2020-01-22 09:32:52,1579685586.46,0.00,1.49,38.00,1579685572.00,103.48,1579685572.00,1579685572.00,2020-01-22 09:32:52.000000
6,3220260,1579685777.78,11100010,2054,151,9,0,1579685803.09,0.00,1.49,38.00,0.00,231.09,1579685803.09,1579685803.09,2020-01-22 09:36:43.094337
7,3220261,1579685902.27,11100010,2054,1225,10,0,1579685942.90,0.00,1.49,38.00,0.00,139.90,1579685942.99,1579685942.99,2020-01-22 09:39:02.993886
8,3220262,1579686153.00,11100010,2054,274,11,2020-01-22 09:42:33,1579686224.48,0.00,1.49,38.00,1579686153.00,281.51,1579686153.00,1579686153.00,2020-01-22 09:42:33.000000
9,3220263,1579686283.44,11100010,2054,273,12,0,1579686249.65,0.00,1.49,38.00,0.00,96.65,1579686249.65,1579686249.65,2020-01-22 09:44:09.646925


In [89]:
%%time
for idx, row in test_df.iterrows():
    timestamp_scaled = row['TIMESTAMP_SCALED']
    if (pd.isna(timestamp_scaled)):
        test_df.at[idx, 'TIMESTAMP_SCALED'] = row['TIMESTAMP_BK']

CPU times: user 3min 24s, sys: 2.22 s, total: 3min 26s
Wall time: 3min 46s


In [90]:
test_df_bk = test_df
test_df = test_df_bk

In [91]:
test_df.head(60)

,index,TIMESTAMP,TIMESTAMP_BK,TIMESTAMP_SCALED,DATETIME_SCALED
0,3220254,1579685016.00,1579685016.00,1579685016.00,2020-01-22 09:23:36.000000
1,3220255,1579685127.33,0.00,1579685137.48,2020-01-22 09:25:37.481148
2,3220256,1579685246.00,1579685246.00,1579685246.00,2020-01-22 09:27:26.000000
3,3220257,1579685337.48,0.00,1579685333.59,2020-01-22 09:28:53.591269
4,3220258,1579685483.07,0.00,1579685473.01,2020-01-22 09:31:13.008599
5,3220259,1579685572.00,1579685572.00,1579685572.00,2020-01-22 09:32:52.000000
6,3220260,1579685803.09,0.00,1579685777.78,2020-01-22 09:36:17.779127
7,3220261,1579685942.90,0.00,1579685902.27,2020-01-22 09:38:22.273511
8,3220262,1579686153.00,1579686153.00,1579686153.00,2020-01-22 09:42:33.000000
9,3220263,1579686249.65,0.00,1579686283.44,2020-01-22 09:44:43.438368


In [92]:
test_df = test_df[['index', 'TIMESTAMP_SCALED', 'TIMESTAMP_BK']].rename(columns={"TIMESTAMP_SCALED": "TIMESTAMP"})

In [93]:
test_df.head(50)

,index,TIMESTAMP,TIMESTAMP_BK
0,3220254,1579685016.00,1579685016.00
1,3220255,1579685137.48,0.00
2,3220256,1579685246.00,1579685246.00
3,3220257,1579685333.59,0.00
4,3220258,1579685473.01,0.00
5,3220259,1579685572.00,1579685572.00
6,3220260,1579685777.78,0.00
7,3220261,1579685902.27,0.00
8,3220262,1579686153.00,1579686153.00
9,3220263,1579686283.44,0.00


In [94]:
# test_df.to_csv("./out/test_df_timestamp_adjusted.csv")
test_df[['index', 'TIMESTAMP']].to_csv("./out/output_ready_6_rescaled.csv", index=False)

In [49]:
# test_2 = pd.read_csv("./out/output_ready_best.csv")

In [51]:
# pd.merge(test_2, test_df, on='index', how='inner')

,index,TIMESTAMP_x,TIMESTAMP_y,TIMESTAMP_BK
0,3220254,1579685016.00,1579685016.00,1579685016.00
1,3220255,1579685137.05,1579685131.00,0.00
2,3220256,1579685246.00,1579685246.00,1579685246.00
3,3220257,1579685339.27,1579685341.34,0.00
4,3220258,1579685475.31,1579685475.20,0.00
...,...,...,...,...
1557459,4777713,1579883967.00,1579883967.00,1579883967.00
1557460,4777714,1579886880.00,1579886880.00,1579886880.00
1557461,4777715,1579887223.60,1579887932.62,0.00
1557462,4777716,1579887676.35,1579887983.28,0.00
